# Liriscat paper experiments
### 1. Init
#### 1.1. Import libraries (necessary)

In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["PYTHONHASHSEED"] = "0"
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ":4096:8"

import liriscat
liriscat.utils.set_seed(0)

import logging
import gc
import json
import torch
liriscat.utils.set_seed(0)
import pandas as pd
from importlib import reload

#### 1.2. Set up the loggers (recommended)

In [2]:
liriscat.utils.setuplogger(verbose = True, log_name="liriscat", debug=False)

### 2. CDM prediction
#### 2.1. Training and testing, sequential version

In [3]:
import warnings

gc.collect()
torch.cuda.empty_cache()

In [4]:
config = liriscat.utils.generate_eval_config(load_params=True, esc = 'error', valid_metric= 'mi_acc', pred_metrics = ["mi_acc"], profile_metrics = ['meta_doa'], save_params=False, n_query=6, num_epochs=100, batch_size=512)
liriscat.utils.set_seed(config["seed"])

config["dataset_name"] = "math2"
logging.info(config["dataset_name"])
config['learning_rate'] = 0.0001
config['inner_user_lr'] = 0.016848380924625605
config['lambda'] = 9.972254466547545e-06

config['meta_lr'] = 0.5
config['patience'] = 20#5
config['num_inner_users_epochs'] = 3
config['d_in'] = 4
config['num_responses'] = 12

#pred_metrics,df_interp = test(config)

CUDA is available. Using GPU.
[INFO 21:04] math2


In [5]:
logging.info(f'#### {config["dataset_name"]} ####')
logging.info(f'#### config : {config} ####')
config['embs_path']='../embs/'
config['params_path']='../ckpt/'

gc.collect()
torch.cuda.empty_cache()

# Dataset downloading for doa and rm
warnings.filterwarnings("ignore", message="invalid value encountered in divide")
warnings.filterwarnings("ignore", category=RuntimeWarning)

## Concept map format : {question_id : [category_id1, category_id2, ...]}
concept_map = json.load(open(f'../datasets/2-preprocessed_data/{config["dataset_name"]}_concept_map.json', 'r'))
concept_map = {int(k): [int(x) for x in v] for k, v in concept_map.items()}

## Metadata map format : {"num_user_id": ..., "num_item_id": ..., "num_dimension_id": ...}
metadata = json.load(open(f'../datasets/2-preprocessed_data/{config["dataset_name"]}_metadata.json', 'r'))


## Tensor containing the nb of modalities per question
nb_modalities = torch.load(f'../datasets/2-preprocessed_data/{config["dataset_name"]}_nb_modalities.pkl',weights_only=True)


[INFO 21:04] #### math2 ####
[INFO 21:05] #### config : {'seed': 0, 'dataset_name': 'math2', 'load_params': True, 'save_params': False, 'embs_path': '../embs/', 'params_path': '../ckpt/', 'early_stopping': True, 'esc': 'error', 'verbose_early_stopping': False, 'disable_tqdm': False, 'valid_metric': 'mi_acc', 'learning_rate': 0.0001, 'batch_size': 512, 'valid_batch_size': 10000, 'num_epochs': 100, 'eval_freq': 1, 'patience': 20, 'device': device(type='cuda'), 'lambda': 9.972254466547545e-06, 'tensorboard': False, 'flush_freq': True, 'pred_metrics': ['mi_acc'], 'profile_metrics': ['meta_doa'], 'num_responses': 12, 'low_mem': False, 'n_query': 6, 'CDM': 'impact', 'i_fold': 0, 'num_inner_users_epochs': 3, 'num_inner_epochs': 10, 'inner_lr': 0.0001, 'inner_user_lr': 0.016848380924625605, 'meta_lr': 0.5, 'meta_trainer': 'Adam', 'num_workers': 0, 'pin_memory': False, 'debug': False, 'd_in': 4} ####


In [6]:
meta_trainers = ['Approx_GAP']
for meta_trainer in meta_trainers : 
    config['meta_trainer'] = meta_trainer
    logging.info(f'#### meta_trainer : {config["meta_trainer"]} ####')
    for i_fold in range(1,2) : 
        config['i_fold'] = i_fold
            
        logging.info(f'#### i_fold : {i_fold} ####')
        ## Dataframe columns : (user_id, question_id, response, category_id)
        train_df = pd.read_csv(
            f'../datasets/2-preprocessed_data/{config["dataset_name"]}_train_{i_fold}.csv',
            encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                                    "dimension_id": int})
        valid_df = pd.read_csv(
            f'../datasets/2-preprocessed_data/{config["dataset_name"]}_valid_{i_fold}.csv',
            encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                                    "dimension_id": int})
        test_df = pd.read_csv(
            f'../datasets/2-preprocessed_data/{config["dataset_name"]}_test_{i_fold}.csv',
            encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                                    "dimension_id": int})

        train_data = liriscat.dataset.CATDataset(train_df, concept_map, metadata, config,nb_modalities)
        valid_data = liriscat.dataset.EvalDataset(valid_df, concept_map, metadata, config,nb_modalities)
        test_data = liriscat.dataset.EvalDataset(test_df, concept_map, metadata, config,nb_modalities)

        for seed in range(1) :
            config['seed'] = seed
            logging.info(f'#### seed : {seed} ####')

            train_data.reset_rng()
            valid_data.reset_rng()
            test_data.reset_rng()

            S = liriscat.selectionStrategy.Random(metadata,**config)
            S.init_models(train_data, valid_data)
            S.train(train_data, valid_data)
            liriscat.utils.set_seed(0)
            S.reset_rng()
            d = (S.evaluate_test(test_data, train_data, valid_data))
            logging.info(d)
            logging.info(liriscat.utils.pareto_index(d))

    torch.cuda.empty_cache()

[INFO 21:05] #### meta_trainer : Approx_GAP ####
[INFO 21:05] #### i_fold : 1 ####
[INFO 21:15] #### seed : 0 ####
[INFO 21:15] Random_cont_model
[INFO 21:15] compiling CDM model
[INFO 21:17] compiling selection model
[INFO 21:17] ------- START Training
[INFO 21:17] train on cuda


/home/arthur/liriscat/liriscat/selectionStrategy/random.py:30: UserWarning: get_params() Notimplemented
  warnings.warn('get_params() Notimplemented')
  0%|          | 0/100 [00:00<?, ?it/s]

[INFO 21:18] ----- User batch : 0
[INFO 21:23] ----- User batch : 1
[INFO 21:26] ----- User batch : 2
[INFO 21:29] ----- User batch : 3
[INFO 21:32] ----- User batch : 4
[INFO 21:34] - meta_params gn: [0.32099825143814087]
[INFO 21:34] - meta_params norm: [34.032470703125]
[INFO 21:34] - meta_params: 0.5
[INFO 21:41] rmse : 0.5917553305625916
[INFO 21:41] valid_rmse : 0.7567203640937805
[INFO 21:41] valid_loss : 0.9675600528717041


  1%|          | 1/100 [00:23<38:27, 23.31s/it]

[INFO 21:41] ----- User batch : 0
[INFO 21:45] ----- User batch : 1
[INFO 21:48] ----- User batch : 2
[INFO 21:52] ----- User batch : 3
[INFO 21:55] ----- User batch : 4
[INFO 21:57] - meta_params gn: [0.30382025241851807]
[INFO 21:57] - meta_params norm: [34.774452209472656]
[INFO 21:57] - meta_params: 0.5
[INFO 22:02] rmse : 0.5900689363479614
[INFO 22:02] valid_rmse : 0.7545638680458069
[INFO 22:02] valid_loss : 0.9673460721969604


...

[INFO 31:41] ----- User batch : 0
[INFO 31:45] ----- User batch : 1
[INFO 31:48] ----- User batch : 2
[INFO 31:52] ----- User batch : 3
[INFO 31:55] ----- User batch : 4
[INFO 31:57] - meta_params gn: [0.17030194401741028]
[INFO 31:57] - meta_params norm: [46.03704833984375]
[INFO 31:57] - meta_params: 0.0078125
[INFO 32:01] rmse : 0.584810197353363
[INFO 32:01] valid_rmse : 0.747839093208313
[INFO 32:01] valid_loss : 0.9649723768234253


 29%|██▉       | 29/100 [10:43<26:15, 22.20s/it]

[INFO 32:01] -- END Training --
[INFO 32:01] train on cuda
Learning users emb shape: torch.Size([3911, 16])



100%|██████████| 6/6 [00:04<00:00,  1.42it/s]


[INFO 32:12] ({0: {'mi_acc': 0.635077178478241}, 1: {'mi_acc': 0.6367712616920471}, 2: {'mi_acc': 0.642551064491272}, 3: {'mi_acc': 0.6440458297729492}, 4: {'mi_acc': 0.651419997215271}, 5: {'mi_acc': 0.653911292552948}}, {0: {'meta_doa': 0.5062931470726217}, 1: {'meta_doa': 0.5107671239843226}, 2: {'meta_doa': 0.5167609327968822}, 3: {'meta_doa': 0.5239388405435574}, 4: {'meta_doa': 0.5335979268072653}, 5: {'meta_doa': 0.543656102709063}})
[INFO 32:12] 0.01996683782378124


In [ ]:
config['meta_trainer'] = 'Approx_GAP'
#config['meta_trainer'] = 'Adam'
for seed in range(1) :
    config['seed'] = seed
    logging.info(f'#### seed : {seed} ####')
    
    train_data.reset_rng()
    valid_data.reset_rng()
    test_data.reset_rng()


    S = liriscat.selectionStrategy.Random(metadata,**config)
    S.init_models(train_data, valid_data)
    S.train(train_data, valid_data)
    liriscat.utils.set_seed(0)
    S.reset_rng()
    d = (S.evaluate_test(test_data, train_data, valid_data))
    print(liriscat.utils.pareto_index(d))
    print(d)
    torch.cuda.empty_cache()